In [10]:
# https://www.kaggle.com/prateekmaj21/tf-idf-in-python

In [1]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer
from numpy import dot
from numpy.linalg import norm

# Задание 1. Реализовать tfidf датасета

In [12]:
import os
data = list(open(os.path.join('data', 'quora.txt'), encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [43]:
count = CountVectorizer()
word_count = count.fit_transform(data[:1000])
word_count.shape

(1000, 3069)

In [44]:
tfidf_transformer=TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=count.get_feature_names_out(),columns=["idf_weights"])

In [45]:
#inverse document frequency
df_idf.sort_values(by=['idf_weights']).head()

,idf_weights
what,1.960720
the,1.995252
is,2.131102
how,2.332806
in,2.423958


In [46]:
#tfidf
tf_idf_vector = tfidf_transformer.transform(word_count)
feature_names = count.get_feature_names_out()
first_document_vector=tf_idf_vector[1]
df_tfifd = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df_tfifd.sort_values(by=["tfidf"],ascending=False).head()

,tfidf
fast,0.440432
addiction,0.440432
ways,0.440432
overcome,0.398123
food,0.355814


# Задание 2. Находим косинусное расстояние по вектору tfidf

In [48]:
max_cos = 0
v1 = None
v2 = None
pos1 = None
pos2 = None
for i in range(word_count.shape[0]):
    for j in range(i+1, word_count.shape[0]-1):
        cos = dot(tf_idf_vector[i].todense(), tf_idf_vector[j].T.todense())/(norm(tf_idf_vector[i].T.todense())*norm(tf_idf_vector[j].T.todense()))
        if max_cos < cos:
            max_cos = cos
            v1 = tf_idf_vector[i]
            v2 = tf_idf_vector[j]
            pos1 = i
            pos2 = j
print(max_cos)
print(data[pos1], data[pos2], sep='\n')

[[0.82402803]]
What universities does Wabash National recruit new grads from? What majors are they looking for?

What universities does PetMeds recruit new grads from? What majors are they looking for?



# Задание 3. Находим самые схожие предложения по вектору из BoW

In [51]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input
    and returns a vector of size of filtered_vocab.It puts 0 if the
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    '''This functions returns a list in which the order remains
    same and no item repeats.Using the set() function does not
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

In [55]:
tokenizer = WordPunctTokenizer()
data_lower = [i.lower() for i in data[:1000]]
data_tok = [tokenizer.tokenize(i) for i in data_lower]

In [56]:
all_words = []
for i in data_tok:
    for j in i:
        all_words.append(j)

In [57]:
vocab = unique(all_words)

In [58]:
filtered_vocab = []
for w in vocab:
    if w not in stopwords and w not in special_char:
        filtered_vocab.append(w)

In [59]:
vector_of_data_tok = []
for i in data_tok:
    vector_of_data_tok.append(vectorize(i))

In [60]:
max_cos = 0
v1 = None
v2 = None
pos1 = None
pos2 = None
for i in range(len(vector_of_data_tok)):
    for j in range(i+1, len(vector_of_data_tok)-1):
        cos = dot(vector_of_data_tok[i], vector_of_data_tok[j])/(norm(vector_of_data_tok[i])*norm(vector_of_data_tok[j]))
        if max_cos < cos:
            max_cos = cos
            v1 = vector_of_data_tok[i]
            v2 = vector_of_data_tok[j]
            pos1 = i
            pos2 = j
print(max_cos)
print(data_tok[pos1], data_tok[pos2], sep='\n')

0.9095085938862486
['what', 'universities', 'does', 'wabash', 'national', 'recruit', 'new', 'grads', 'from', '?', 'what', 'majors', 'are', 'they', 'looking', 'for', '?']
['what', 'universities', 'does', 'petmeds', 'recruit', 'new', 'grads', 'from', '?', 'what', 'majors', 'are', 'they', 'looking', 'for', '?']


In [61]:
print(data[pos1], data[pos2], sep='\n')

What universities does Wabash National recruit new grads from? What majors are they looking for?

What universities does PetMeds recruit new grads from? What majors are they looking for?



**Вывод:** TF-IDF и BoW справились одинаково хорошо с поиском похожих предложений. Косинусная мера при применении BoW выше